In [ ]:
#!pip install import-ipynb
import pandas as pd
import numpy as np
import requests as rq
import import_ipynb
import util as u
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', None)
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
def get_senadores():    
    url = "https://legis.senado.leg.br/dadosabertos/senador/lista/atual"
    df_senadores = u.parse_XML(url,["CodigoParlamentar","NomeParlamentar","SiglaPartidoParlamentar","UfParlamentar"],'Parlamentares/Parlamentar/IdentificacaoParlamentar')
    df_senadores = df_senadores.set_index("CodigoParlamentar")
    return df_senadores

In [ ]:
def get_relatorias_do_senador(codigo_parlamentar):
    url = "https://legis.senado.leg.br/dadosabertos/senador/%d/relatorias" % codigo_parlamentar
    df = u.parse_XML(url, ["Materia/Codigo","Materia/DescricaoIdentificacao","Comissao/Sigla","CodigoTipoRelator","DataDesignacao","DataDestituicao","DescricaoMotivoDestituicao", "DescricaoTipoRelator"]
,"Parlamentar/Relatorias/Relatoria")
    df.rename(columns = {'Materia/Codigo' : 'CodigoMateria','Materia/DescricaoIdentificacao':'Identificacao','Comissao/Sigla' :'LocalRelatoria'}, inplace = True)
    
    df['CodigoParlamentar'] = codigo_parlamentar
    df['CodigoMateria'] = pd.to_numeric(df['CodigoMateria'])
    df['DataDesignacao'] = pd.to_datetime(df['DataDesignacao'])
    df['DataDestituicao'] = pd.to_datetime(df['DataDestituicao'])
    df['DataDesignacao'] = (df['DataDesignacao']
                          .astype(str) # <- cast to string to simplify
                                       #    .replace() in newer versions
                          .replace({'NaT': None} # <- replace with None
                         ))
    df['DataDestituicao'] = (df['DataDestituicao']
                          .astype(str) # <- cast to string to simplify
                                       #    .replace() in newer versions
                          .replace({'NaT': None} # <- replace with None
                         ))
    return df  
    

In [ ]:
def get_relatorias_de_senadores(senadores_a_serem_pesquisados):    
    relatorias_atualizadas = pd.DataFrame()    
    for codigoParlamentar, sen in senadores_a_serem_pesquisados.iterrows():            
        relatorias_do_senador = get_relatorias_do_senador(int(codigoParlamentar))    
        relatorias_atualizadas = pd.concat([relatorias_atualizadas,relatorias_do_senador])
    
    return relatorias_atualizadas

In [ ]:
def get_autorias_do_senador(codigo_parlamentar):    
    url = "https://legis.senado.leg.br/dadosabertos/senador/%d/autorias" % codigo_parlamentar
    df = u.parse_XML(url, ["Materia/Codigo","Materia/DescricaoIdentificacao","Materia/Ementa","Materia/Data","IndicadorAutorPrincipal"]
    ,"Parlamentar/Autorias/Autoria")
    df.rename(columns = {'Materia/Codigo' : 'CodigoMateria'}, inplace = True)
    df.rename(columns = {'Materia/DescricaoIdentificacao' : 'Identificacao'}, inplace = True)
    df.rename(columns = {'Materia/Ementa' : 'Ementa'}, inplace = True)
    df.rename(columns = {'Materia/Data' : 'Data'}, inplace = True)
    df['CodigoParlamentar'] = codigo_parlamentar
    df['CodigoMateria'] = pd.to_numeric(df['CodigoMateria'])
    df['Data'] = pd.to_datetime(df['Data'])
    df['Data'] = (df['Data']
                          .astype(str) # <- cast to string to simplify
                                       #    .replace() in newer versions
                          .replace({'NaT': None} # <- replace with None
                         ))
    return df  

In [ ]:
def get_autorias_dos_senadores(senadores_a_serem_pesquisados):    
    autorias_dos_senadores = pd.DataFrame()    
    for codigoParlamentar, sen in senadores_a_serem_pesquisados.iterrows():            
        autorias_do_senador = get_autorias_do_senador(int(codigoParlamentar))    
        autorias_dos_senadores = pd.concat([autorias_dos_senadores,autorias_do_senador])
    
    return autorias_dos_senadores

In [ ]:
def get_url_discursos(codigo_parlamentar, ano):
    dataInicio = str(ano) + "0101"
    dataFim = str(ano) + "1231"
    urlDiscursos = "https://legis.senado.leg.br/dadosabertos/senador/"+str(codigo_parlamentar)+"/discursos?dataInicio="+dataInicio+"%d&dataFim="+dataFim
    df = u.parse_XML(urlDiscursos,["CodigoPronunciamento","UrlTexto"],"Parlamentar/Pronunciamentos/Pronunciamento")
    df = df.set_index('CodigoPronunciamento')
    return df

In [ ]:
def get_discurso(url): 
    page = rq.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    texto = soup.find(class_='texto-integral')
    if not texto:
        return None
    else:
        return limpar_texto(texto.get_text())

In [ ]:
def limpar_texto(texto):
    lixo_carac = ['\n','\t', '\r', '\xa0', 'â\x80\x93']
    for char in lixo_carac:
        texto = texto.replace(char, '')

    return texto

In [ ]:
def get_discursos(parlamentares_a_serem_pesquisados,ano_inicio,ano_fim):
    df_discursos = pd.DataFrame()
    for codigo, row in parlamentares_a_serem_pesquisados.iterrows():    
        try: 
            for ano in range(ano_inicio, ano_fim):
                df_discurso_sen = get_url_discursos(codigo,ano)
                df_discurso_sen['texto'] = df_discurso_sen['UrlTexto'].apply(get_discurso)
                df_discurso_sen['CodigoParlamentar'] = codigo
                df_discursos = pd.concat([df_discursos,df_discurso_sen])
            #materias_legislatura_atual.at[codigo, 'PegouDetalhe'] = True    
        except TimeoutError:
            print ("não achei %s" % codigo)
    return df_discursos

In [ ]:
def get_interesse_relatoria(data_designacao, data_destituicao, motivo_destituicao,descricao_tipo_relator, local_relatoria):
    valor_interesse = 0
    justificativa_interesse = None
    #Começa calculando há quantos anos foi designado como relator
    hoje = datetime.today()
    data_designacao_format =  datetime.strptime(data_designacao, '%Y-%m-%d')
    anos_relatoria = relativedelta(hoje, data_designacao_format).years
    
    #Atribui o interesse
    if (not data_destituicao):
        #É o relator atual
        if (anos_relatoria<1):
            valor_interesse =  4
            justificativa_interesse = "Relator atual designado há menos de 1 ano em " + local_relatoria
        elif (anos_relatoria<3):
            valor_interesse =  3
            justificativa_interesse = "Relator atual designado há menos de 3 anos em " + local_relatoria
        else:
            valor_interesse =  2
            justificativa_interesse = "Relator atual designado há mais de 3 anos em " + local_relatoria
    
    #Não é o relator atual    
    if (motivo_destituicao == 'Redistribuição'):
        valor_interesse =  -5
        justificativa_interesse = "Foi relator mas houve redistribuição da relatoria em " + local_relatoria
    if (descricao_tipo_relator == 'Relator Ad hoc'):
        valor_interesse =  1     
        justificativa_interesse = "Foi relator ad hoc em " + local_relatoria
    if (motivo_destituicao == 'Deliberação da matéria' or motivo_destituicao == 'Matéria deliberada no plenário'
       or motivo_destituicao == 'Parecer de Plenário' or motivo_destituicao == 'Parecer Oferecido'
       ):
        data_destituicao_format = datetime.strptime(data_designacao, '%Y-%m-%d')
        anos_destituicao = relativedelta(hoje, data_destituicao_format).years
        if (anos_destituicao<1):
            valor_interesse =  7
            justificativa_interesse = "Relatou a matéria há menos de 1 ano em " + local_relatoria
        elif (anos_destituicao<3):
            valor_interesse =  6
            justificativa_interesse = "Relatou a matéria há menos de 3 anos em " + local_relatoria
        else:
            valor_interesse =  5
            justificativa_interesse = "Relatou a matéria há mais de 3 anos em " + local_relatoria
    
    #É atribuído 0 quando a Matéria é prejudicada, ou seja, a relatoria não se efetivou de fato
    if (justificativa_interesse == None):
        valor_interesse = 0
        justificativa_interesse = "Foi designado como relator mas a matéria foi prejudicada em " + local_relatoria
    
    return pd.Series([valor_interesse, justificativa_interesse])

In [ ]:
def get_interesse_autoria(indicador_autor_principal,data, identificacao):
    valor_interesse = 0
    justificativa_interesse = None
    hoje = datetime.today()
    data_format =  datetime.strptime(data, '%Y-%m-%d')
    anos_autoria = relativedelta(hoje, data_format).years
    if (indicador_autor_principal == "Sim"):
        if (anos_autoria<1):
            valor_interesse = 7
            justificativa_interesse = "Autor principal da matéria há menos de 1 ano"
        elif (anos_autoria<3):
            valor_interesse = 6
            justificativa_interesse = "Autor principal da matéria há menos de 3 anos"
        else:
            valor_interesse = 5        
            justificativa_interesse = "Autor principal da matéria há mais de 3 ano"
    
    else:
        #não é autor principal
        if (identificacao.startswith('PEC')):
            valor_interesse = 0
            justificativa_interesse = "Co-autor de PEC"
        else:
            if (anos_autoria<1):
                valor_interesse = 5
                justificativa_interesse = "Co-autor da matéria há menos de 1 ano"
            elif (anos_autoria<3):
                valor_interesse = 4
                justificativa_interesse = "Co-autor da matéria há menos de 3 anos"
            else:
                valor_interesse = 3
                justificativa_interesse = "Co-autor da matéria há mais de 3 anos"
    #return valor_interesse,justificativa_interesse
    return pd.Series([valor_interesse, justificativa_interesse])
        

In [ ]:
def get_interesse_emenda(data):
    valor_interesse = 0
    justificativa_interesse = None
    hoje = datetime.today()
    data_format =  datetime.strptime(data, '%Y-%m-%d')
    anos_autoria = relativedelta(hoje, data_format).years    
    if (anos_autoria<1):
        valor_interesse = 7
        justificativa_interesse = "Autor de emenda há menos de 1 ano"
    elif (anos_autoria<3):
        valor_interesse = 6
        justificativa_interesse = "Autor de emenda há menos de 3 anos"
    else:
        valor_interesse = 5        
        justificativa_interesse = "Autor de emenda há mais de 3 anos"
    
    
    #return valor_interesse,justificativa_interesse
    return pd.Series([valor_interesse, justificativa_interesse])
        

In [ ]:
def get_interesse_discurso(data,tipoDescricao):
    valor_interesse = 0
    justificativa_interesse = None
    hoje = datetime.today()
    data_format =  datetime.strptime(data, '%Y-%m-%d')
    anos_discurso = relativedelta(hoje, data_format).years
    if (anos_discurso<1):
        valor_interesse = 5
        justificativa_interesse = "Discursou sobre a matéria ("+tipoDescricao+") há menos de 1 ano"
    elif (anos_discurso < 3):
        valor_interesse = 4
        justificativa_interesse = "Discursou sobre a matéria ("+tipoDescricao+") há menos de 3 anos"
    else:
        valor_interesse = 3
        justificativa_interesse = "Discursou sobre a matéria ("+tipoDescricao+") há mais de 3 anos"
    
    return pd.Series([valor_interesse, justificativa_interesse])
        

### Carrega os senadores da legislatura

In [ ]:
#Recupera os senadores da 57ª legislatura
df_senadores = get_senadores()

In [ ]:
df_senadores

In [ ]:
df_senadores.to_csv('df_senadores.csv')

### Relatorias dos Senadores

In [ ]:
# Recupera as relatorias dos senadores
df_relatorias_senadores = get_relatorias_de_senadores(df_senadores) 
df_relatorias_senadores = df_relatorias_senadores[df_relatorias_senadores['Identificacao'].str.startswith("P")]
#Considera apenas as autorias desta legislatura e da anterir
#df_relatorias_senadores = df_relatorias_senadores.query('DataDesignacao >= "2019-01-01" or DataDestituicao >= "2019-01-01"')

In [ ]:
df_relatorias_senadores

In [ ]:
df_relatorias_senadores = df_relatorias_senadores.set_index('CodigoMateria')
len(df_relatorias_senadores)

In [ ]:
#Atribui o nível de interesse do Senador naquela matéria
df_relatorias_senadores[['interesse','justificativa_interesse']] = df_relatorias_senadores.apply(
    lambda row: get_interesse_relatoria(row['DataDesignacao'], row['DataDestituicao'],row['DescricaoMotivoDestituicao'],row['DescricaoTipoRelator'],row['LocalRelatoria']),axis = 1)

In [ ]:
df_relatorias_senadores.to_csv('df_relatorias.csv')

### Autorias dos Senadores

In [ ]:
# Recupera as autorias dos senadores
df_autorias_senadores = get_autorias_dos_senadores(df_senadores)  

#Considera apenas as matérias iniciadas com a letra P
df_autorias_senadores = df_autorias_senadores[df_autorias_senadores['Identificacao'].str.startswith("P")]

In [ ]:
df_autorias_senadores

In [ ]:
#Atribui o nível de interesse na autoria
df_autorias_senadores[['interesse','justificativa_interesse']] = df_autorias_senadores.apply(
    lambda row: get_interesse_autoria(row['IndicadorAutorPrincipal'],row['Data'],row['Identificacao']),axis = 1)


In [ ]:
df_autorias_senadores = df_autorias_senadores.set_index('CodigoMateria')
len(df_autorias_senadores)

In [ ]:
df_autorias_senadores.to_csv('df_autorias.csv')

In [ ]:
df_autorias_senadores

## Discursos dos senadores

In [ ]:
# Recupera os discursos dos senadores
df_discursos_senadores = pd.read_csv('df_discursos.csv')
df_discursos_senadores.rename(columns={'codigoMateria': 'CodigoMateria','codigoParlamentar' : 'CodigoParlamentar'}, inplace=True)

In [ ]:
#Atribui o nível de interesse nos discursos
df_discursos_senadores[['interesse','justificativa_interesse']] = df_discursos_senadores.apply(
    lambda row: get_interesse_discurso(row['dataDiscurso'],row['tipoDescricao']),axis = 1)


In [ ]:
df_discursos_senadores

In [ ]:
df_discursos_senadores = df_discursos_senadores.set_index('CodigoMateria')
len(df_discursos_senadores)

In [ ]:
df_discursos_senadores

## Emendas dos Senadores

In [ ]:
# Recupera os discursos dos senadores
df_emendas_senadores = pd.read_csv('df_emendas.csv')

In [ ]:
df_emendas_senadores

In [ ]:
df_emendas_senadores = df_emendas_senadores[df_emendas_senadores['CodigoParlamentar'].notna()]

In [ ]:
df_emendas_senadores = df_emendas_senadores[df_emendas_senadores.CodigoParlamentar.astype(int).isin(df_senadores.index.astype(int))]

In [ ]:
#Atribui o nível de interesse nas emendas
df_emendas_senadores[['interesse','justificativa_interesse']] = df_emendas_senadores.apply(
    lambda row: get_interesse_emenda(row['DataApresentacao']),axis = 1)

In [ ]:
df_emendas_senadores = df_emendas_senadores.set_index('CodigoMateria')
len(df_emendas_senadores)

In [ ]:
df_emendas_senadores

### Juntando conjuntos de dados de interesse do senador (relatorias, autorias, discursos e emendas)

In [ ]:
df_interesse = pd.concat([df_autorias_senadores[['CodigoParlamentar','interesse','justificativa_interesse']], 
                          df_relatorias_senadores[['CodigoParlamentar','interesse','justificativa_interesse']], 
                          df_discursos_senadores[['CodigoParlamentar','interesse','justificativa_interesse']],
                         df_emendas_senadores[['CodigoParlamentar','interesse','justificativa_interesse']]])
df_interesse = df_interesse.reset_index()
df_interesse = df_interesse.drop_duplicates(subset=['CodigoParlamentar', 'CodigoMateria', 'interesse', 'justificativa_interesse'])

In [ ]:
import pandas as pd

# Suponha que você tenha um DataFrame chamado df_interesse

# Defina uma função personalizada para concatenar as justificativas
def concatenar_justificativa(series):
    return ' - '.join(series)

# Crie um dicionário com as funções de agregação para cada coluna
aggregation_functions = {
    'interesse': 'sum',  # Somar os valores de Interesse
    'justificativa_interesse': concatenar_justificativa  # Concatenar as justificativas
}

# Faça o group by por CodigoParlamentar e CodigoMateria e aplique as funções de agregação
df_resultante = df_interesse.groupby(['CodigoParlamentar', 'CodigoMateria']).agg(aggregation_functions).reset_index()


In [ ]:
df_resultante

In [ ]:
#df_resultante.to_csv('df_resultado_interesse.csv')
df_resultante.to_csv('interesse.csv')

## Atualização

In [ ]:
df_senadores = get_senadores()

# Recupera as relatorias dos senadores
df_relatorias_senadores = get_relatorias_de_senadores(df_senadores) 

# Recupera as autorias dos senadores
df_autorias_senadores = get_autorias_dos_senadores(df_senadores)  

# Combine os DataFrames df_relatorias_senadores e df_autorias_senadores
df_combinado = pd.concat([df_relatorias_senadores, df_autorias_senadores])
df_combinado.to_csv('df_combinado.csv')
